In [1]:
import requests
import bs4
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
# 爬ptt政治板
def get_some_page_ptt_data(URL,last_n_page):
    def get_content(soup):
        ## 查找所有html 元素 抓出內容
        main_container = soup.find(id='main-container')
        # 把所有文字都抓出來
        all_text = main_container.text
        # 把整個內容切割透過 "-- " 切割成2個陣列
        pre_text = all_text.split('--')[0]
        # 把每段文字 根據 '\n' 切開
        texts = pre_text.split('\n')
        # 如果你爬多篇你會發現 
        contents = texts[2:]
        # 內容
        content = ''.join(contents)
        return content
    
    # 根據PTT的URL取得這個PTT頁面的資料的DataFrame格式
    def get_this_index_data(URL):
        
        # 發送get 請求 到 ptt 八卦版
        response = requests.get(URL, headers = {'cookie': 'over18=1;'})
        
        #  把網頁程式碼(HTML) 丟入 bs4模組分析
        soup = bs4.BeautifulSoup(response.text,"html.parser")

        ## PTT 上方4個欄位
        result = soup.find_all('div','title')
        urls = []
        titles = []
        contents = []
        for i in result:
            try:
                post_url = 'https://www.ptt.cc'+str(i).split('href="')[1].split('">')[0]
                urls.append(post_url)
                titles.append(str(i).split("]")[1].split("<")[0])
                response = requests.get(post_url, headers = my_headers)
                soup = bs4.BeautifulSoup(response.text,"html.parser")
                contents.append(get_content(soup))
            except:
                pass #(本文已被刪除)
        df = pd.DataFrame()
        df['title'] = titles
        df['url'] = urls
        df['content'] = contents
        return df
    
    # 計算目前ptt政治版總共有多少page
    def get_total_page_number(URL):
        response = requests.get(URL, headers = {'cookie': 'over18=1;'})
        soup = bs4.BeautifulSoup(response.text,"html.parser")
        number = int(str(soup.select('#action-bar-container > div > div.btn-group.btn-group-paging > a:nth-child(2)')[0]).split('index')[1].split('.html')[0])
        return number
    
    total_page_number = get_total_page_number(URL)
    
    # 取得一些政治板page的連結稱為URLS
    URLS = [f"https://www.ptt.cc/bbs/HatePolitics/index{i}.html" for i in range(total_page_number,total_page_number-last_n_page,-1)]
    
    # 根據特定URL取得這個PAGE的資料dataframe格式
    def get_this_index_data(URL):
        # 設定Header與Cookie
        my_headers = {'cookie': 'over18=1;'}
        # 發送get 請求 到 ptt 八卦版
        response = requests.get(URL, headers = my_headers)
        #  把網頁程式碼(HTML) 丟入 bs4模組分析
        soup = bs4.BeautifulSoup(response.text,"html.parser")
        ## PTT 上方4個欄位
        result = soup.find_all('div','title')
        urls = []
        titles = []
        contents = []
        for i in tqdm(result):
            try:
                titles.append(i.text.replace("\n",''))
                urls.append('https://www.ptt.cc'+str(i.find_all('a')[0]['href']))
                response = requests.get(urls[-1], headers = my_headers)
                soup = bs4.BeautifulSoup(response.text,"html.parser")
                contents.append(get_content(soup))
            except:
                print('頁面刪除')
        df = pd.DataFrame()
        min_len = min(len(titles),len(urls),len(contents))
        df['title'] = titles[:min_len]
        df['url'] = urls[:min_len]
        df['content'] = contents[:min_len]
        return df
    
    #取得PTT政治板最新的PAGE
    df = get_this_index_data(URL) 
    for u in tqdm(URLS): #對URLS做遍歷
        df = df.append(get_this_index_data(u))
    return df

In [2]:
URL = "https://www.ptt.cc/bbs/HatePolitics/index.html"
last_n_page = 10
df = get_some_page_ptt_data(URL,last_n_page)

 20%|████████████████▌                                                                  | 4/20 [00:02<00:09,  1.74it/s]

頁面刪除



 35%|█████████████████████████████                                                      | 7/20 [00:03<00:06,  1.88it/s]

頁面刪除



 35%|█████████████████████████████                                                      | 7/20 [00:04<00:07,  1.65it/s]

...


 25%|████████████████████▊                                                              | 5/20 [00:02<00:07,  2.02it/s]

頁面刪除



 55%|█████████████████████████████████████████████                                     | 11/20 [00:05<00:03,  2.29it/s]

頁面刪除



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:57<00:00, 11.78s/it]


In [3]:
df

,title,url,content
0,[討論] 賴執政後的台灣預測,https://www.ptt.cc/bbs/HatePolitics/M.16857671...,國防：兵役 軍備 全民皆兵政策不變美國叫我們做的全都要做外交：繼續斷交外交部：我們努力到最後...
1,[黑特] 這個世界太噁心了,https://www.ptt.cc/bbs/HatePolitics/M.16857673...,噁心的人在說別人噁心 到底是誰的噁心比較噁心 我覺得這個世界都很噁心 看...
2,[討論] 性騷擾案 民進黨又加分了！,https://www.ptt.cc/bbs/HatePolitics/M.16857675...,如題啦！雖然說性騷擾不可取但不得不說民進黨這次處理的還不錯果決 明快 該怎麼辦就怎麼辦最會反...
3,[公告] magic543 舒華我來了 14天,https://www.ptt.cc/bbs/HatePolitics/M.16857675...,
4,[討論] 國民黨會因為這件事情而打贏2024嗎?,https://www.ptt.cc/bbs/HatePolitics/M.16857676...,賴清德好像不打算把林飛帆換下來國民黨的侯友宜有可能因此贏得總統寶座嗎?國民黨的立委有沒有因此...
...,...,...,...
15,Re: [討論] 陳右豪準備開告?,https://www.ptt.cc/bbs/HatePolitics/M.16857277...,性騷擾屬於不容易舉證的事情目前的metoo又都不是現行犯看來幾個男性當事人再跟律師討論過都很...
16,[討論] 明年信賴聯盟，非賴不投,https://www.ptt.cc/bbs/HatePolitics/M.16857282...,這禮拜得了武漢肺炎，都在家裡發燒沒辦法看新聞今天難得有力氣，網路電視一起炸鍋，都在講性騷擾的...
17,Re: [討論] 誰接蔡EE的主席、總統 都會很倒楣吧....,https://www.ptt.cc/bbs/HatePolitics/M.16857283...,※ 引述《llww (開心渡過每一天)》之銘言：: 現在DPP正在爆炸的性平案，就是蔡EE擔...
18,Re: [轉錄] 黃瀞瑩:在你們眼中，我錯在不夠勇敢?,https://www.ptt.cc/bbs/HatePolitics/M.16857288...,https://reurl.cc/2Lo6Mr昨天才有民眾黨護航文指稱 劉嘉仁案為”沒有性騷...


In [4]:
import gc
import re
import time
import warnings
from datetime import datetime, timedelta

# from selenium import webdriver
import bs4
import numpy as np
import pandas as pd
import requests
import streamlit as st
from bs4 import BeautifulSoup
from tqdm import tqdm
from transformers import pipeline

warnings.filterwarnings("ignore")


def predict_function(df, person_name, min_data_n=3):  # 最小資料量min_data_n
    # 合併標題和內文
    df["all_text"] = df["title"] + df["content"]
    idx_lst = []
    # 對每一筆新聞進行遍歷
    for idx, text in enumerate(df["all_text"]):
        text = str(text)
        # 檢查該政治人物是否出現在文章中
        if person_name in text:
            idx_lst.append(idx)
    # 擷取出有出現特定政治人物的df
    person_name_df = df.iloc[idx_lst, :]
    person_name_df = person_name_df.reset_index(drop=True)
    # 計算一共有多少筆談論該政治人物的文章
    st.write(f"出現{person_name}的資料筆數:", len(person_name_df))
    # 資料不足的情況
    if len(person_name_df) < min_data_n:
        st.write(f"資料過少,少於{min_data_n}筆")

    # 評分機制 由特定pretrain model做zero-shot-classification看這則文章屬於["positive", "negative"]哪一種
    st.write("create classifier pipeline")
    person_name_df["情緒"] = 0
    classifier = pipeline(
        task="zero-shot-classification",
        model="joeddav/xlm-roberta-large-xnli",  # joeddav/xlm-roberta-large-xnli有支援中文
        device="cpu",
        use_auth_token=True,
    )
    candidate_labels = ["positive", "negative"]

    # 對文章做遍歷個別去計算情緒分數
    st.write("遍歷爬取到的文章個別去計算情緒分數")
    a_dict = {}
    for idx, text in tqdm(enumerate(person_name_df["all_text"].values.tolist())):
        a_dict[text] = classifier(text, candidate_labels)["labels"][0]
        person_name_df.loc[idx, "情緒"] = classifier(text, candidate_labels)["labels"][0]

    # 最後將score定義為:positive.sum()/negative.sum()
    st.write("對情緒分數取平均")
    if (person_name_df["情緒"] == "negative").sum() > 0:
        score = (person_name_df["情緒"] == "positive").sum() / (
            person_name_df["情緒"] == "negative"
        ).sum()
    # 分母為0無法計算
    else:
        score = "score = (person_name_df['情緒']=='positive').sum()/(person_name_df['情緒']=='negative').sum() but can not caculate"

    return score,a_dict

In [5]:
score,a_dict = predict_function(df.head(100), '蔡英文', min_data_n=3)

2023-06-03 12:55:21.353 
  command:

    streamlit run C:\Users\ricky\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
9it [01:11,  7.98s/it]


In [6]:
score

0.125

In [7]:
a_dict

{'Re: [討論] 為什麼性騷案鬧那麼大，賴神還是很穩？https://i.imgur.com/zHbeE9b.jpghttps://i.imgur.com/6QRZeYI.jpghttps://i.imgur.com/eoSrS79.jpghttps://imgur.com/a/Asp6VKp.jpg至於賴清德 就真的軟軟的這次事件雖然跟賴沒直接關係但凸顯了賴清德對黨的掌控能力差很難讓人相信他有什麼改革的能力如果我是候選人絕對是馬上大刀闊斧改革讓扣分變加分你看賴清德這種咖難怪會鬥輸蔡英文': 'negative',
 '[新聞] 曝范雲、林靜儀反對檢討公務機關性騷機1.新聞網址︰※ 請附上有效原文連結或短網址 ※https://tinyurl.com/2hsjjnw82.新聞來源︰ETtoday新聞雲3.完整新聞標題快訊／曝范雲、林靜儀反對檢討公務機關性騷機制\u3000藍黨團：民進黨病了4.完整新聞內容︰※ 請完整轉載原文 請勿修改內文與刪減 ※記者蘇晏男／台北報導民進黨連日爆發數起受害者自訴曾遭性騷事件，宛如民進黨版的「#MeToo」，外界紛紛籲綠營檢討相關機制。不過，國民黨立法院黨團今（3日）揭露先前提案要求行政院通盤檢討目前公務機關性騷擾申訴流程機制，然而表決時反對人數超過贊成人數未通過，而投下反對者共58人且皆為綠委，包含范雲、林靜儀等；國民黨團總召曾銘宗表示，要請你們出來講清楚、說明白。國民黨團3日下午舉行「賴清德牽起共犯 縱容性騷擾！」記者會，會中曾銘宗提及2023年1月19日審查112年度中央政府總預算案時，鑒於時任原能會主委謝曉星性騷擾案，國民黨團提案要求檢討性騷擾相關法規及相關通報機制。該提案中寫道，「為確保調查過程不受權力不對等關係而影響到調查結果的公信力，要求原能會檢討修正性騷擾申訴調查處理要點之法制上缺漏；並請行政院通盤檢討目前公務機關性騷擾申訴流程機制，須將涉嫌性騷擾之政務人員先予停職再啟動性平調查程序，確實保障被害人之權益」。曾銘宗向記者表示，換句話說就是要防止被吃案的可能。然而立法院會記名表決結果為出席人數94人、贊成36人、反對58人、棄權0人；其中贊成者除國民黨立委外，還有時代力量立委王婉諭與邱顯智，而投反對者皆為民進黨立委。曾銘宗指出，當時投反對票的綠委包含一再宣稱女權至上、維護女權的范雲、林靜儀、吳思瑤、高嘉瑜、邱議瑩等；

In [14]:
for k in a_dict:
    print(k,a_dict[k])
    print('========================')

Re: [討論] 為什麼性騷案鬧那麼大，賴神還是很穩？https://i.imgur.com/zHbeE9b.jpghttps://i.imgur.com/6QRZeYI.jpghttps://i.imgur.com/eoSrS79.jpghttps://imgur.com/a/Asp6VKp.jpg至於賴清德 就真的軟軟的這次事件雖然跟賴沒直接關係但凸顯了賴清德對黨的掌控能力差很難讓人相信他有什麼改革的能力如果我是候選人絕對是馬上大刀闊斧改革讓扣分變加分你看賴清德這種咖難怪會鬥輸蔡英文 negative
[新聞] 曝范雲、林靜儀反對檢討公務機關性騷機1.新聞網址︰※ 請附上有效原文連結或短網址 ※https://tinyurl.com/2hsjjnw82.新聞來源︰ETtoday新聞雲3.完整新聞標題快訊／曝范雲、林靜儀反對檢討公務機關性騷機制　藍黨團：民進黨病了4.完整新聞內容︰※ 請完整轉載原文 請勿修改內文與刪減 ※記者蘇晏男／台北報導民進黨連日爆發數起受害者自訴曾遭性騷事件，宛如民進黨版的「#MeToo」，外界紛紛籲綠營檢討相關機制。不過，國民黨立法院黨團今（3日）揭露先前提案要求行政院通盤檢討目前公務機關性騷擾申訴流程機制，然而表決時反對人數超過贊成人數未通過，而投下反對者共58人且皆為綠委，包含范雲、林靜儀等；國民黨團總召曾銘宗表示，要請你們出來講清楚、說明白。國民黨團3日下午舉行「賴清德牽起共犯 縱容性騷擾！」記者會，會中曾銘宗提及2023年1月19日審查112年度中央政府總預算案時，鑒於時任原能會主委謝曉星性騷擾案，國民黨團提案要求檢討性騷擾相關法規及相關通報機制。該提案中寫道，「為確保調查過程不受權力不對等關係而影響到調查結果的公信力，要求原能會檢討修正性騷擾申訴調查處理要點之法制上缺漏；並請行政院通盤檢討目前公務機關性騷擾申訴流程機制，須將涉嫌性騷擾之政務人員先予停職再啟動性平調查程序，確實保障被害人之權益」。曾銘宗向記者表示，換句話說就是要防止被吃案的可能。然而立法院會記名表決結果為出席人數94人、贊成36人、反對58人、棄權0人；其中贊成者除國民黨立委外，還有時代力量立委王婉諭與邱顯智，而投反對者皆為民進黨立委。曾銘宗指出，當時投反對票的綠委包含一再宣稱女權至上、維護女權的范雲、林靜儀、吳思瑤、高嘉瑜、邱議瑩等；他要問問所有綠委，連防治性騷